# Single Training on GPU 0

# Quick Links <a name = "Top"></a>

<ol>
<li><a href = #setup>Begin Training</a></li>
</ol>

# Imports

In [1]:
import os

#os.environ["CUDA_VISIBLE_DEVICES"]='0'

#print('Current Conda Environment: {}'.format(os.environ['CONDA_DEFAULT_ENV']))

In [2]:
import talos as ta
from talos.model import lr_normalizer, early_stopper, hidden_layers

import tensorflow as tf
  
available_gpus = tf.config.experimental.list_physical_devices('GPU')
built_with_cuda = tf.test.is_built_with_cuda()

if not (not available_gpus) & built_with_cuda:
    print("The installed version of TensorFlow {} includes GPU support.\n".format(tf.__version__))
    print("Num GPUs Available: ", len(available_gpus), "\n")
else:
    print("The installed version of TensorFlow {} does not include GPU support.\n".format(tf.__version__))
    
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

from tensorflow.compat.v1.keras import callbacks, backend as K
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.utils import multi_gpu_model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.optimizers import Adam, Nadam, RMSprop, SGD, Adagrad
from tensorflow.keras.layers import ReLU, LeakyReLU

from datetime import datetime
import pandas as pd
import numpy as np
import shutil

from copy import deepcopy

import time

from numpy.random import seed
seed(1)
tf.random.set_seed(1)

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True
config.gpu_options.per_process_gpu_memory_fraction = 0.99
sess = tf.compat.v1.Session(config = config)
K.set_session(sess)

The installed version of TensorFlow 2.3.1 does not include GPU support.

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2005844600110865474
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 99545989670981413
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 7648989804002239458
physical_device_desc: "device: XLA_GPU device"
]


# Hilfsfunktionen

### Enum für Training-Set

In [3]:
from enum import Enum

class TrainingSet(Enum):
    SYNTHETIC = 1
    REAL = 2
    MIXED = 3

### Output Directory

* <i>SSD</i>, falls genug Speicher auf SSD im SymLink <i>fast_output</i> verfügbar ist
* <i>HDD</i>, falls möglicherweise zu wenig SSD-Speicher verfügbar ist $\rightarrow$ <i>output</i>

In [4]:
from enum import IntEnum

class OutputDirectory(IntEnum):
    HDD = 0
    SSD = 1
    
output_path = ['output', 'fast_output']

### Benutzerdefinierte Kostenfunktion & Metrik

In [5]:
def circular_mse(y_true, y_pred):
    max_error = tf.constant(360, dtype='float32')
    return K.mean(K.square(K.minimum(K.abs(y_pred - y_true), max_error - K.abs(y_pred - y_true))), axis = -1)

def circular_mae(y_true, y_pred):
    max_error = tf.constant(360, dtype='float32')
    return K.mean(K.minimum(K.abs(y_pred - y_true), K.abs(max_error - K.abs(y_pred - y_true))), axis = -1)

def custom_mae(y_true, y_pred):
    return K.mean(K.abs(y_pred - y_true), axis = -1)

### Convert Label_Type into suitable label names.
$\Rightarrow$ Angular / Normalized $\rightarrow$ ['Elevation', 'Azimuth']

$\Rightarrow$ Stereographic $\rightarrow$ ['S_x', 'S_y']

In [6]:
def get_Label_Names(label_type):
    if label_type == 'Angular' or label_type == 'Normalized':
        return ['Elevation', 'Azimuth']
    elif label_type == 'Stereographic':
        return ['S_x', 'S_y']
    else:
        assert(True, 'LabelType Invalid')
        return None

### Convert String into Reduction Metric Function

In [7]:
def get_Reduction_Metric(metric):
    
    if metric == 'custom_mae':
        return [custom_mae]
    elif metric == 'tf.keras.metrics.MeanAbsoluteError()':
        return [tf.keras.metrics.MeanAbsoluteError()]
    elif metric == 'circular_mae':
        return [circular_mae]
    elif metric == 'mean_squared_error':
        return ['mean_squared_error']
    else:
        assert(False, 'Metric yet unknown - Please modify get_Reduction_Metric to meet your requirements')
        return None

### Automatische Optimizer Generierung aus String

In [8]:
def make_optimizer(optimizer):
    # [Adam, Nadam, Adagrad, RMSprop]
    if optimizer == "<class 'keras.optimizers.Adam'>":
        return Adam
    elif optimizer == "<class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>":
        return Adam
    elif optimizer == "<class 'keras.optimizers.Nadam'>":
        return Nadam
    elif optimizer == "<class 'keras.optimizers.Adagard'>":
        return Adagard
    elif optimizer == "<class 'keras.optimizers.RMSprop'>":
        return RMSprop
    else:
        print('ERROR::: Unspecified Optimizer')

### Trainingsset-Typ nach String Converter

In [9]:
def trainingset_to_string(ts):
    if ts == TrainingSet.SYNTHETIC:
        return 'Synth'
    elif ts == TrainingSet.REAL:
        return 'Real'
    elif ts == TrainingSet.MIXED:
        return 'Mixed'
    else:
        print('Unknown TrainingSet')
        return None

### Generierung Datenpipeline (Angepasst für Talos)

In [10]:
def create_data(batch_size, num_samples, label_type):
    # if Block für synthetische Daten, um nur auf realen Daten zu trainieren _USE_SYNTHETIC_TRAIN_DATA
    # 1. lege df_train und df_valid als leere Liste an
    # 2. If-block um Zeile df = ... bis df_valid
    
    if trainingset == TrainingSet.SYNTHETIC:
        df = pd.read_csv(_CSV_FILE)
        df_shuffled = df.sample(frac = 1, random_state = 1)
        df_train = df_shuffled[0 : int(num_samples * 0.8 // batch_size * batch_size)]
        df_valid = df_shuffled.drop(df_shuffled.index[0 : df_train.shape[0]])[0 : int(num_samples * 0.2 // batch_size * batch_size)]
        
    elif trainingset == TrainingSet.MIXED:
        df = pd.read_csv(_CSV_FILE)
        df_shuffled = df.sample(frac = 1, random_state = 1)
        df_train = df_shuffled[0 : int(num_samples * 0.8 // batch_size * batch_size)]
        df_valid = df_shuffled.drop(df_shuffled.index[0 : df_train.shape[0]])[0 : int(num_samples * 0.2 // batch_size * batch_size)]
        
        df_real = pd.read_csv(_CSV_FILE_REAL)
        df_shuffled_real = df_real.sample(frac = 1, random_state = 1)
        df_shuffled_real = df_shuffled_real.drop(df_shuffled_real.index[(df_shuffled_real.shape[0] - 61) : df_shuffled_real.shape[0]])
        df_train_real = df_shuffled_real[0: int(df_shuffled_real.shape[0] * 0.8 // batch_size * batch_size)]   
        df_valid_real = df_shuffled_real.drop(df_shuffled_real.index[0 : df_train_real.shape[0]])
        df_train = df_train.drop(df_train.index[df_train.shape[0] - df_train_real.shape[0] : df_train.shape[0]])
        df_valid = df_valid.drop(df_valid.index[df_valid.shape[0] - df_valid_real.shape[0] : df_valid.shape[0]])
        df_train = df_train.append(df_train_real)
        df_valid= df_valid.append(df_valid_real)
    
    elif trainingset == TrainingSet.REAL: # Add check for num_samples, once the real dataset increases
        df_real = pd.read_csv(_CSV_FILE_REAL)
        df_shuffled_real = df_real.sample(frac = 1, random_state = 1)
        df_shuffled_real = df_shuffled_real.drop(df_shuffled_real.index[(df_shuffled_real.shape[0] - 61) : df_shuffled_real.shape[0]])
        df_train = df_shuffled_real[0 : int(df_shuffled_real.shape[0] * 0.8 // batch_size * batch_size)]   
        df_valid = df_shuffled_real.drop(df_shuffled_real.index[0 : df_train.shape[0]])
        
    else:
        print('Create_Data :: should not have reached here')
        

        
    if _USE_DATA_AUGMENTATION:
        train_data_generator = ImageDataGenerator(
            rescale = 1./255,
            width_shift_range = 0.1,
            height_shift_range = 0.1,
            zoom_range = 0.1,
            brightness_range = (0.25, 0.75),
            fill_mode = 'nearest'
        )
    else:
        train_data_generator = ImageDataGenerator(
            rescale = 1./255
        )
        
    print('Y-Col: {}'.format(get_Label_Names(label_type)))
    print('Train Data Generator: ', end = '')
    
    train_generator = train_data_generator.flow_from_dataframe(
        dataframe = df_train,
        directory = _IMAGE_DIR,
        x_col = 'Filename',
        y_col = get_Label_Names(label_type),
        class_mode = 'raw',
        target_size = (224, 224),
        color_mode = 'rgb',
        shuffle = True,
        seed = 77,
        batch_size = batch_size
    )
    
    valid_data_generator = ImageDataGenerator(
        rescale = 1./255
    )
    
    print('Validation Data Generator: ', end = '')
    
    valid_generator = valid_data_generator.flow_from_dataframe(
        dataframe = df_valid,
        directory = _IMAGE_DIR,
        x_col = 'Filename',
        y_col = get_Label_Names(label_type),
        class_mode = 'raw',
        target_size = (224, 224),
        color_mode = 'rgb',
        shuffle = False,
        seed = 77,
        batch_size = batch_size
    )
    
    return train_generator, valid_generator

### Generierung Modell (Angepasst für Talos)

In [11]:
def grid_model_fine(x, y, x_val, y_val, params):
    
    K.clear_session()
    
    train_generator, valid_generator = create_data(params['batch_size'], params['samples'], params['label_type'])
    tg_steps_per_epoch = train_generator.n // train_generator.batch_size
    vg_validation_steps = valid_generator.n // valid_generator.batch_size
    print('Steps per Epoch: {}, Validation Steps: {}'.format(tg_steps_per_epoch, vg_validation_steps))
    
    dropout_rate = params['dropout']
    first_neuron = params['first_neuron']
    
    if params['activation'] == 'leakyrelu':
        activation_layer = LeakyReLU(alpha = params['leaky_alpha'])
    elif params['activation'] == 'relu':
        activation_layer = ReLU()
    
    model = Sequential()
    cnn = VGG16(weights = 'imagenet', include_top = False, input_shape = (224, 224, 3))
    
    for layer in cnn.layers[:15]:
        layer.trainable = False
        #print(layer.name, layer.trainable)
        
    print('_________________________________________________________________')
    print('{:>16} {:>16}'.format('Network Layer', 'Trainable'))
    print('=================================================================')
    for layer in cnn.layers:
        print('{:>16} {:>16}'.format(layer.name, layer.trainable))
    print('_________________________________________________________________\n')
    
    model.add(cnn)
    
    fc = Sequential()
    fc.add(Flatten(input_shape = model.output_shape[1:])) # (7, 7, 512)
    
    fc.add(Dense(units = first_neuron, kernel_initializer = glorot_uniform(seed = 1)))
    fc.add(activation_layer)
    if dropout_rate > 0.0:
        fc.add(Dropout(rate = dropout_rate))
    
    print('Number Hidden Layers {}'.format(params['hidden_layers']))
    hidden_neuron_fraction = first_neuron
    for i in range(params['hidden_layers']):
        hidden_neuron_fraction = hidden_neuron_fraction // 2
        fc.add(Dense(units = hidden_neuron_fraction, kernel_initializer = glorot_uniform(seed = 1)))
        fc.add(activation_layer)
        if dropout_rate > 0.0:
            fc.add(Dropout(rate = dropout_rate))
    
    fc.add(Dense(units = 2, kernel_initializer = glorot_uniform(seed = 1)))
    fc.load_weights(_MODEL_DIR + _MODEL_TO_LOAD)
    model.add(fc)
    print('Fully Connected Layers added to Base Network')
    
    print('Using Loss: {} \nand Reduction Metric: {}'.format(
        params['loss_function'], 
        get_Reduction_Metric(params['reduction_metric'])))
    
    model.compile(
        #optimizer=params['optimizer'](lr=lr_normalizer(params['lr'], params['optimizer'])*1e-2),
        optimizer = params['optimizer'](lr = lr_normalizer(params['lr'], params['optimizer']) * 1e-3),
        loss = params['loss_function'],
        metrics = get_Reduction_Metric(params['reduction_metric'])
    )
    print('Model was compiled')
    print(model.summary())
    print('_________________________________________________________________')
    
    checkpointer = callbacks.ModelCheckpoint(
        filepath = _LOG_DIR + 'CNN_Base_{}_Model_and_Weights_{}.hdf5'.format(_MODEL_TO_LOAD_INDEX, train_generator.n),
        monitor =  params['monitor_value'],
        verbose = 1,
        save_weights_only = False,
        save_best_only = True,
        mode = 'min'
    )
    print('Checkpointer was created')
    
    csv_logger = callbacks.CSVLogger(
        filename = _LOG_DIR + 'CNN_Base_{}_Logger_{}.csv'.format(_MODEL_TO_LOAD_INDEX, train_generator.n),
        separator = ',',
        append = False
    )
    print('CSV Logger was created')

    lr_reducer = callbacks.ReduceLROnPlateau(
        monitor = 'val_loss',
        factor = 0.1,
        patience = 13,
        verbose = 1,
        mode = 'min',
        min_delta = 0.0001
    )
    print('Learning Rate Reducer was created')
    
    early_stopper = callbacks.EarlyStopping(
        monitor = 'val_loss',
        min_delta = 0,
        #patience = 15,
        patience = 20,
        verbose = 1,
        mode = 'min',
        restore_best_weights = True
    )
    print('Early Stopper was created')
    
    out = model.fit(
        x = train_generator,
        steps_per_epoch = tg_steps_per_epoch,
        validation_data = valid_generator,
        validation_steps = vg_validation_steps,
        callbacks = [checkpointer, csv_logger, lr_reducer, early_stopper],
        epochs = params['epochs'],
        workers = 8
    )
    
    return out, model

# Feinoptimierung <a name = "setup"></a><a href = #Top>Up</a></p>

#### Hyper Parameter

In [12]:
#     Adam = RMSprop + Momentum (lr=0.001)
#     Nadam = Adam RMSprop + Nesterov-Momentum (lr=0.002)
#     RMSprop = (lr=0.001)
#     SGD = (lr=0.01)
#     Adagrad

global_hyper_parameter = {
    'samples': None,
    'epochs': None,
    'batch_size': None,
    'optimizer': None,
    'lr': None,
    'first_neuron': None,
    'dropout': None,
    'activation': None,
    'leaky_alpha': None,
    'hidden_layers': None,
    # beginning from here, Values should only contain one single entry:
    # ===============================================================
    'label_type': ['Normalized'], # Stereographic, Angular, Normalized
    'loss_function': None,
    'reduction_metric': None,
    'monitor_value': None
}


### Training Setup 

In [13]:
_RUN = 'SYNTH'
_LOSS = 'MSE'
_DATASET_NAME = '2020-05-28'
_DEVICE = 'TITAN_GPU0'

storage = OutputDirectory.SSD # 'fast_output' if ssd storage may suffice, 'output' otherwise

if global_hyper_parameter['label_type'][0] == 'Stereographic':
    _CSV_SYNTH_FILE_NAME = 'images_synthetisch_stereographic.csv'
    _CSV_REAL_FILE_NAME = 'images_real_stereographic.csv'
    
elif global_hyper_parameter['label_type'][0] == 'Angular':
    _CSV_SYNTH_FILE_NAME = 'images_synthetisch.csv'
    _CSV_REAL_FILE_NAME = 'images_real.csv'
    
elif global_hyper_parameter['label_type'][0] == 'Normalized':
    _CSV_SYNTH_FILE_NAME = 'images_synthetisch_normalized.csv'
    _CSV_REAL_FILE_NAME = 'images_real_normalized.csv'
    
else:
    assert(True, 'Label Type Invalid')

In [14]:
trainingset = TrainingSet.SYNTHETIC
_USE_DATA_AUGMENTATION = True

In [15]:
_IMAGE_DIR = '..\\dataset_mm\\{}\\'.format(_DATASET_NAME)
_CSV_FILE = _IMAGE_DIR + _CSV_SYNTH_FILE_NAME
_CSV_FILE_REAL = _IMAGE_DIR + _CSV_REAL_FILE_NAME

_note = '_Custom-MAE'

_MODEL_DIR = '..\\output\\{}_Regression_{}\\{}_{}_Base{}\\'.format(_RUN, _LOSS, _DATASET_NAME, global_hyper_parameter['label_type'][0], _note)
_NET_DIR = '{}_Regression_{}\\{}_{}_Top_1{}\\{}_TD\\'.format(_RUN, _LOSS, _DATASET_NAME, global_hyper_parameter['label_type'][0], _note, trainingset_to_string(trainingset))
_LOG_DIR = '..\\{}\\{}'.format(output_path[storage], _NET_DIR)

if(not os.path.exists(_LOG_DIR)):
    os.makedirs(_LOG_DIR)
else:
    input('Directory >>| {} |<< existiert bereits. Fortsetzen auf eigene Gefahr! (Weiter mit Enter)'.format(_LOG_DIR))

device_file = open(_LOG_DIR + '{}.txt'.format(_DEVICE), "a+")

### Top 3 FC-Gewichte

In [16]:
base_results = _MODEL_DIR + '..\\{}_{}_Base{}_Results.csv'.format(_DATASET_NAME, global_hyper_parameter['label_type'][0], _note)
df = pd.read_csv(base_results).drop(columns = ['round_epochs', 'samples', 'epochs'], axis = 0)
sort_value = df['monitor_value'][0]
df = df.sort_values(sort_value, axis = 0, ascending = True, inplace = False, kind = 'quicksort', na_position = 'last')
print('Displaying: {}'.format(base_results))
df.head(10)

Displaying: ..\output\SYNTH_Regression_MSE\2020-05-28_Normalized_Base_Custom-MAE\..\2020-05-28_Normalized_Base_Custom-MAE_Results.csv


,Unnamed: 0,start,end,duration,loss,custom_mae,val_loss,val_custom_mae,activation,batch_size,dropout,first_neuron,hidden_layers,label_type,loss_function,lr,monitor_value,optimizer,reduction_metric
4,4,12/09/20-093859,12/09/20-093940,41.077978,2.714420,0.479261,0.116606,0.242770,relu,32,0.25,1024,0,Normalized,mean_squared_error,1,val_custom_mae,<class 'tensorflow.python.keras.optimizer_v2.a...,custom_mae
6,6,12/09/20-094022,12/09/20-094046,23.500743,5.638185,0.679268,0.121281,0.245946,relu,64,0.25,1024,0,Normalized,mean_squared_error,1,val_custom_mae,<class 'tensorflow.python.keras.optimizer_v2.a...,custom_mae
5,5,12/09/20-093941,12/09/20-094022,41.251921,2.443644,0.525243,0.133008,0.270280,relu,32,0.50,1024,0,Normalized,mean_squared_error,1,val_custom_mae,<class 'tensorflow.python.keras.optimizer_v2.a...,custom_mae
7,7,12/09/20-094046,12/09/20-094110,23.965750,5.211095,0.725741,0.142764,0.282519,relu,64,0.50,1024,0,Normalized,mean_squared_error,1,val_custom_mae,<class 'tensorflow.python.keras.optimizer_v2.a...,custom_mae
1,1,12/09/20-092309,12/09/20-092350,41.071499,7.006359,1.277110,0.158711,0.302616,leakyrelu,32,0.50,1024,0,Normalized,mean_squared_error,1,val_custom_mae,<class 'tensorflow.python.keras.optimizer_v2.a...,custom_mae
2,2,12/09/20-092351,12/09/20-093835,884.509856,12.850731,1.252772,0.168039,0.316974,leakyrelu,64,0.25,1024,0,Normalized,mean_squared_error,1,val_custom_mae,<class 'tensorflow.python.keras.optimizer_v2.a...,custom_mae
0,0,12/09/20-090723,12/09/20-092309,946.496623,6.604711,0.991875,0.167170,0.319055,leakyrelu,32,0.25,1024,0,Normalized,mean_squared_error,1,val_custom_mae,<class 'tensorflow.python.keras.optimizer_v2.a...,custom_mae
3,3,12/09/20-093836,12/09/20-093859,23.120344,13.420115,1.614795,0.179288,0.325059,leakyrelu,64,0.50,1024,0,Normalized,mean_squared_error,1,val_custom_mae,<class 'tensorflow.python.keras.optimizer_v2.a...,custom_mae


### GridSerach

In [17]:
def get_params(top_results_index):
    
    #     Adam = RMSprop + Momentum (lr=0.001)
    #     Nadam = Adam RMSprop + Nesterov-Momentum (lr=0.002)
    #     RMSprop = (lr=0.001)
    #     SGD = (lr=0.01)
    #     Adagrad

    hyper_parameter = global_hyper_parameter

    hyper_parameter['samples'] = [100000]
    hyper_parameter['epochs'] = [400]
    hyper_parameter['batch_size'] = [df.iloc[top_results_index]['batch_size']]
    hyper_parameter['optimizer'] = [make_optimizer(df.loc[top_results_index]['optimizer'])]
    hyper_parameter['lr'] = [df.iloc[top_results_index]['lr']]
    hyper_parameter['first_neuron'] = [df.iloc[top_results_index]['first_neuron']]
    hyper_parameter['dropout'] = [df.iloc[top_results_index]['dropout']]
    hyper_parameter['activation'] = [df.iloc[top_results_index]['activation']]
    hyper_parameter['leaky_alpha'] = [0.1] #Default bei LeakyReLU, sonst PReLU
    hyper_parameter['hidden_layers'] = [df.iloc[top_results_index]['hidden_layers']]
    
    hyper_parameter['loss_function'] = [df.iloc[top_results_index]['loss_function']]
    hyper_parameter['reduction_metric'] = [df.iloc[top_results_index]['reduction_metric']]
    hyper_parameter['monitor_value'] = [df.iloc[top_results_index]['monitor_value']]

    return hyper_parameter

### Start Talos

In [ ]:
dummy_x = np.empty((1, 2, 3, 224, 224))
dummy_y = np.empty((1, 2))

with tf.device('/device:GPU:0'):
    #for top_results_index in range(3):
    for top_results_index in [0, 1]:
        #top_results_index = 1
        _MODEL_TO_LOAD_INDEX = df.iloc[top_results_index].name
        _MODEL_TO_LOAD = 'Best_Weights_FC_{}.hdf5'.format(_MODEL_TO_LOAD_INDEX)

        _TMP_DIR = '..\\TMP_TALOS_{}'.format(_DEVICE)
        _CSV_RESULTS = _LOG_DIR + 'Talos_Results_Fine_Idx{}.csv'.format(_MODEL_TO_LOAD_INDEX)

        startTime = datetime.now()
        
        parameters = get_params(top_results_index)

        t = ta.Scan(
            x = dummy_x,
            y = dummy_y,
            model = grid_model_fine,
            params = parameters,
            experiment_name = _TMP_DIR,
            #shuffle=False,
            reduction_metric = parameters['reduction_metric'][0],
            disable_progress_bar = False,
            print_params = True,
            clear_session = True
        )

        print("Time taken:", datetime.now() - startTime)
        
        print('Writing Device File')
        device_file.write('Trained Model: {}'.format(_MODEL_TO_LOAD))

        df_experiment_results = pd.read_csv(_TMP_DIR + '\\' + os.listdir(_TMP_DIR)[0])
        df_experiment_results['Base'] = None
        for i in range(df_experiment_results.shape[0]):
            df_experiment_results['Base'][i] = _MODEL_TO_LOAD_INDEX

        if os.path.isfile(_CSV_RESULTS):
            df_experiment_results.to_csv(_CSV_RESULTS, mode = 'a', index = False, header = False)
        else:
            df_experiment_results.to_csv(_CSV_RESULTS, index = False)

        shutil.rmtree(_TMP_DIR)

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

{'activation': 'relu', 'batch_size': 32, 'dropout': 0.25, 'epochs': 400, 'first_neuron': 1024, 'hidden_layers': 0, 'label_type': 'Normalized', 'leaky_alpha': 0.1, 'loss_function': 'mean_squared_error', 'lr': 1, 'monitor_value': 'val_custom_mae', 'optimizer': <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 'reduction_metric': 'custom_mae', 'samples': 100000}
Y-Col: ['Elevation', 'Azimuth']
Train Data Generator: Found 80000 validated image filenames.
Validation Data Generator: Found 20000 validated image filenames.
Steps per Epoch: 2500, Validation Steps: 625
_________________________________________________________________
   Network Layer        Trainable
         input_1                0
    block1_conv1                0
    block1_conv2                0
     block1_pool                0
    block2_conv1                0
    block2_conv2                0
     block2_pool                0
    block3_conv1                0
    block3_conv2                0
    block3_conv3    

 164/2500 [>.............................] - ETA: 1s - loss: 0.6106 - custom_mae: 0.64 - ETA: 43:48 - loss: 0.5494 - custom_mae: 0.611 - ETA: 58:28 - loss: 0.5525 - custom_mae: 0.615 - ETA: 1:06:02 - loss: 0.4929 - custom_mae: 0.574 - ETA: 1:10:33 - loss: 0.5146 - custom_mae: 0.590 - ETA: 1:13:24 - loss: 0.5076 - custom_mae: 0.585 - ETA: 1:15:32 - loss: 0.5099 - custom_mae: 0.584 - ETA: 1:17:00 - loss: 0.4940 - custom_mae: 0.572 - ETA: 1:18:12 - loss: 0.4923 - custom_mae: 0.574 - ETA: 1:19:12 - loss: 0.4988 - custom_mae: 0.577 - ETA: 1:19:57 - loss: 0.5117 - custom_mae: 0.586 - ETA: 1:20:35 - loss: 0.5107 - custom_mae: 0.586 - ETA: 1:21:06 - loss: 0.5124 - custom_mae: 0.590 - ETA: 1:21:35 - loss: 0.5145 - custom_mae: 0.594 - ETA: 1:22:00 - loss: 0.5096 - custom_mae: 0.593 - ETA: 1:22:20 - loss: 0.5067 - custom_mae: 0.591 - ETA: 1:22:34 - loss: 0.5070 - custom_mae: 0.592 - ETA: 1:22:50 - loss: 0.5017 - custom_mae: 0.589 - ETA: 1:23:02 - loss: 0.4963 - custom_mae: 0.586 - ETA: 1:23:15 - 

 327/2500 [==>...........................] - ETA: 1:22:03 - loss: 0.3900 - custom_mae: 0.521 - ETA: 1:22:01 - loss: 0.3899 - custom_mae: 0.521 - ETA: 1:21:59 - loss: 0.3893 - custom_mae: 0.520 - ETA: 1:21:57 - loss: 0.3893 - custom_mae: 0.521 - ETA: 1:21:55 - loss: 0.3894 - custom_mae: 0.521 - ETA: 1:21:52 - loss: 0.3894 - custom_mae: 0.521 - ETA: 1:21:50 - loss: 0.3892 - custom_mae: 0.520 - ETA: 1:21:48 - loss: 0.3889 - custom_mae: 0.520 - ETA: 1:21:47 - loss: 0.3884 - custom_mae: 0.520 - ETA: 1:21:45 - loss: 0.3889 - custom_mae: 0.520 - ETA: 1:21:43 - loss: 0.3890 - custom_mae: 0.521 - ETA: 1:21:41 - loss: 0.3886 - custom_mae: 0.520 - ETA: 1:21:39 - loss: 0.3882 - custom_mae: 0.520 - ETA: 1:21:37 - loss: 0.3876 - custom_mae: 0.519 - ETA: 1:21:35 - loss: 0.3875 - custom_mae: 0.519 - ETA: 1:21:33 - loss: 0.3876 - custom_mae: 0.520 - ETA: 1:21:31 - loss: 0.3868 - custom_mae: 0.519 - ETA: 1:21:29 - loss: 0.3869 - custom_mae: 0.519 - ETA: 1:21:27 - loss: 0.3864 - custom_mae: 0.519 - ETA: 

 490/2500 [====>.........................] - ETA: 1:16:32 - loss: 0.3504 - custom_mae: 0.495 - ETA: 1:16:29 - loss: 0.3503 - custom_mae: 0.495 - ETA: 1:16:27 - loss: 0.3501 - custom_mae: 0.495 - ETA: 1:16:25 - loss: 0.3500 - custom_mae: 0.495 - ETA: 1:16:23 - loss: 0.3498 - custom_mae: 0.495 - ETA: 1:16:21 - loss: 0.3495 - custom_mae: 0.494 - ETA: 1:16:19 - loss: 0.3494 - custom_mae: 0.494 - ETA: 1:16:17 - loss: 0.3492 - custom_mae: 0.494 - ETA: 1:16:15 - loss: 0.3491 - custom_mae: 0.494 - ETA: 1:16:13 - loss: 0.3490 - custom_mae: 0.494 - ETA: 1:16:11 - loss: 0.3491 - custom_mae: 0.494 - ETA: 1:16:09 - loss: 0.3490 - custom_mae: 0.494 - ETA: 1:16:06 - loss: 0.3490 - custom_mae: 0.494 - ETA: 1:16:04 - loss: 0.3488 - custom_mae: 0.494 - ETA: 1:16:02 - loss: 0.3487 - custom_mae: 0.494 - ETA: 1:16:00 - loss: 0.3486 - custom_mae: 0.494 - ETA: 1:15:58 - loss: 0.3485 - custom_mae: 0.494 - ETA: 1:15:56 - loss: 0.3484 - custom_mae: 0.494 - ETA: 1:15:54 - loss: 0.3482 - custom_mae: 0.494 - ETA: 

...

 988/2500 [==========>...................] - ETA: 58:41 - loss: 0.1907 - custom_mae: 0.345 - ETA: 58:39 - loss: 0.1906 - custom_mae: 0.345 - ETA: 58:37 - loss: 0.1907 - custom_mae: 0.345 - ETA: 58:34 - loss: 0.1907 - custom_mae: 0.345 - ETA: 58:32 - loss: 0.1908 - custom_mae: 0.345 - ETA: 58:30 - loss: 0.1907 - custom_mae: 0.345 - ETA: 58:28 - loss: 0.1907 - custom_mae: 0.345 - ETA: 58:26 - loss: 0.1907 - custom_mae: 0.344 - ETA: 58:24 - loss: 0.1906 - custom_mae: 0.344 - ETA: 58:22 - loss: 0.1906 - custom_mae: 0.344 - ETA: 58:20 - loss: 0.1906 - custom_mae: 0.344 - ETA: 58:18 - loss: 0.1905 - custom_mae: 0.344 - ETA: 58:16 - loss: 0.1906 - custom_mae: 0.344 - ETA: 58:14 - loss: 0.1905 - custom_mae: 0.344 - ETA: 58:12 - loss: 0.1906 - custom_mae: 0.344 - ETA: 58:09 - loss: 0.1906 - custom_mae: 0.344 - ETA: 58:07 - loss: 0.1906 - custom_mae: 0.344 - ETA: 58:05 - loss: 0.1906 - custom_mae: 0.344 - ETA: 58:03 - loss: 0.1906 - custom_mae: 0.344 - ETA: 58:01 - loss: 0.1905 - custom_mae: 0.3

1158/2500 [============>.................] - ETA: 52:45 - loss: 0.1892 - custom_mae: 0.343 - ETA: 52:43 - loss: 0.1892 - custom_mae: 0.343 - ETA: 52:41 - loss: 0.1892 - custom_mae: 0.343 - ETA: 52:38 - loss: 0.1892 - custom_mae: 0.343 - ETA: 52:36 - loss: 0.1892 - custom_mae: 0.343 - ETA: 52:34 - loss: 0.1893 - custom_mae: 0.343 - ETA: 52:32 - loss: 0.1892 - custom_mae: 0.343 - ETA: 52:30 - loss: 0.1892 - custom_mae: 0.343 - ETA: 52:28 - loss: 0.1892 - custom_mae: 0.343 - ETA: 52:26 - loss: 0.1892 - custom_mae: 0.343 - ETA: 52:24 - loss: 0.1892 - custom_mae: 0.343 - ETA: 52:22 - loss: 0.1892 - custom_mae: 0.343 - ETA: 52:20 - loss: 0.1892 - custom_mae: 0.343 - ETA: 52:18 - loss: 0.1892 - custom_mae: 0.343 - ETA: 52:16 - loss: 0.1892 - custom_mae: 0.343 - ETA: 52:13 - loss: 0.1892 - custom_mae: 0.343 - ETA: 52:11 - loss: 0.1892 - custom_mae: 0.343 - ETA: 52:09 - loss: 0.1891 - custom_mae: 0.342 - ETA: 52:07 - loss: 0.1891 - custom_mae: 0.342 - ETA: 52:05 - loss: 0.1891 - custom_mae: 0.3

1328/2500 [==============>...............] - ETA: 46:49 - loss: 0.1881 - custom_mae: 0.341 - ETA: 46:47 - loss: 0.1882 - custom_mae: 0.341 - ETA: 46:45 - loss: 0.1881 - custom_mae: 0.341 - ETA: 46:43 - loss: 0.1881 - custom_mae: 0.341 - ETA: 46:41 - loss: 0.1881 - custom_mae: 0.341 - ETA: 46:38 - loss: 0.1881 - custom_mae: 0.341 - ETA: 46:36 - loss: 0.1881 - custom_mae: 0.341 - ETA: 46:34 - loss: 0.1881 - custom_mae: 0.341 - ETA: 46:32 - loss: 0.1880 - custom_mae: 0.341 - ETA: 46:30 - loss: 0.1880 - custom_mae: 0.341 - ETA: 46:28 - loss: 0.1880 - custom_mae: 0.341 - ETA: 46:26 - loss: 0.1880 - custom_mae: 0.341 - ETA: 46:24 - loss: 0.1880 - custom_mae: 0.341 - ETA: 46:22 - loss: 0.1880 - custom_mae: 0.341 - ETA: 46:20 - loss: 0.1880 - custom_mae: 0.341 - ETA: 46:18 - loss: 0.1880 - custom_mae: 0.341 - ETA: 46:15 - loss: 0.1880 - custom_mae: 0.341 - ETA: 46:13 - loss: 0.1879 - custom_mae: 0.341 - ETA: 46:11 - loss: 0.1879 - custom_mae: 0.341 - ETA: 46:09 - loss: 0.1879 - custom_mae: 0.3

1498/2500 [================>.............] - ETA: 40:53 - loss: 0.1861 - custom_mae: 0.338 - ETA: 40:51 - loss: 0.1861 - custom_mae: 0.338 - ETA: 40:49 - loss: 0.1861 - custom_mae: 0.338 - ETA: 40:47 - loss: 0.1861 - custom_mae: 0.338 - ETA: 40:45 - loss: 0.1860 - custom_mae: 0.338 - ETA: 40:43 - loss: 0.1860 - custom_mae: 0.338 - ETA: 40:41 - loss: 0.1860 - custom_mae: 0.338 - ETA: 40:39 - loss: 0.1860 - custom_mae: 0.338 - ETA: 40:37 - loss: 0.1860 - custom_mae: 0.338 - ETA: 40:35 - loss: 0.1859 - custom_mae: 0.338 - ETA: 40:32 - loss: 0.1859 - custom_mae: 0.338 - ETA: 40:30 - loss: 0.1859 - custom_mae: 0.338 - ETA: 40:28 - loss: 0.1859 - custom_mae: 0.338 - ETA: 40:26 - loss: 0.1859 - custom_mae: 0.338 - ETA: 40:24 - loss: 0.1858 - custom_mae: 0.338 - ETA: 40:22 - loss: 0.1858 - custom_mae: 0.338 - ETA: 40:20 - loss: 0.1858 - custom_mae: 0.338 - ETA: 40:18 - loss: 0.1858 - custom_mae: 0.338 - ETA: 40:16 - loss: 0.1857 - custom_mae: 0.338 - ETA: 40:14 - loss: 0.1858 - custom_mae: 0.3

1606/2500 [==================>...........] - ETA: 34:57 - loss: 0.1841 - custom_mae: 0.336 - ETA: 34:55 - loss: 0.1840 - custom_mae: 0.336 - ETA: 34:53 - loss: 0.1841 - custom_mae: 0.336 - ETA: 34:51 - loss: 0.1840 - custom_mae: 0.336 - ETA: 34:49 - loss: 0.1841 - custom_mae: 0.336 - ETA: 34:47 - loss: 0.1841 - custom_mae: 0.336 - ETA: 34:45 - loss: 0.1841 - custom_mae: 0.336 - ETA: 34:43 - loss: 0.1841 - custom_mae: 0.336 - ETA: 34:40 - loss: 0.1841 - custom_mae: 0.336 - ETA: 34:38 - loss: 0.1841 - custom_mae: 0.336 - ETA: 34:36 - loss: 0.1840 - custom_mae: 0.336 - ETA: 34:34 - loss: 0.1841 - custom_mae: 0.336 - ETA: 34:32 - loss: 0.1840 - custom_mae: 0.336 - ETA: 34:30 - loss: 0.1840 - custom_mae: 0.336 - ETA: 34:28 - loss: 0.1840 - custom_mae: 0.336 - ETA: 34:26 - loss: 0.1840 - custom_mae: 0.336 - ETA: 34:24 - loss: 0.1840 - custom_mae: 0.336 - ETA: 34:22 - loss: 0.1840 - custom_mae: 0.336 - ETA: 34:20 - loss: 0.1841 - custom_mae: 0.336 - ETA: 34:18 - loss: 0.1840 - custom_mae: 0.3

In [ ]:
device_file.close()

# Copy Results to NAS if SSD Directory was selected

In [ ]:
def copy_directory(src, dst, symlinks = False, ignore = None):
    maxLen = 0
    message = ''        
    
    if not os.path.exists(dst):
        
        message = 'Creating Path: {}'.format(src)
        maxLen = max(maxLen, len(message))
        print(message + ' ' * (maxLen - len(message)), end = '\r')
        
        os.makedirs(dst)
        
    for item in os.listdir(src):
        
        s = os.path.join(src, item)
        d = os.path.join(dst, item)
        
        if os.path.isdir(s):
            
            message = 'Copying Directory: {}'.format(s)
            maxLen = max(maxLen, len(message))
            print(message + ' ' * (maxLen - len(message)), end = '\r')
            
            shutil.copytree(s, d, symlinks, ignore)
            
        else:
            
            if not os.path.exists(d): #or os.stat(s).st_mtime - os.stat(d).st_mtime > 1:
                
                message = 'Copying File: {}'.format(s)
                maxLen = max(maxLen, len(message))
                print(message + ' ' * (maxLen - len(message)), end = '\r')
                
                shutil.copy2(s, d)
        
        time.sleep(.5)
     
    message = 'Coyping... Done'
    maxLen = max(maxLen, len(message))
    print(message + ' ' * (maxLen - len(message)), end = '\n')

def delete_directory(src, terminator = '\n'):
    message = ''
    maxLen = 0
    
    try:
        message = 'Deleting {}'.format(src)
        maxLen = max(maxLen, len(message))
        print(message + ' ' * (maxLen - len(message)), end = '\r')
        
        shutil.rmtree(src)
        
    except OSError as e:
        message = 'Error: {} : {}'.format(src, e.strerror)
        maxLen = max(maxLen, len(message))
        print(message + ' ' * (maxLen - len(message)), end = '\n')
        return
    
    message = 'Deleting... Done'
    maxLen = max(maxLen, len(message))
    print(message + ' ' * (maxLen - len(message)), end = terminator)

    
def copy_fine_training(src, dst):
    copy_directory(src, dst)
    delete_directory(src, terminator = '\r')
    delete_directory(src + '..\\', terminator = '\r')
    if not os.listdir(src + '..\\..\\'):
        delete_directory(src + '..\\..\\', terminator = '\r')

In [ ]:
if(storage == OutputDirectory.SSD):
    _COPY_DIR = '..\\output\\{}'.format(_NET_DIR)
    copy_fine_training(_LOG_DIR, _COPY_DIR)

# <a name = "CMSE.Mixed"></a><a href = #Top>Up</a></p>